<table>
   <tr>     
    <td><img src="./images/logo-qiskit.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left"></td>
    <td>  </td>
    <td><img src="./images/logo-IBM.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left"></td>
   </tr>
</table>
<br>

# <center>Welcome </center>

<div class="alert alert-block alert-info">

# Using qiskit and IBM backends
</div>

1. build a circuit
2. execute with `qasm_simulator`
3. execute with `statevector_simulator`
4. execute on quantum backend


### Setup

- Install qiskit using the installation guide in qiskit.org https://qiskit.org/documentation/getting_started.html

(it is recommended to use Python virtual environments (`venv`) and Jupyter Notebooks). 


Verify the following cell runs error free : 


In [ ]:
import qiskit
d = qiskit.__qiskit_version__

for k in d.keys():
    print(f"{k:25} : {d[k]}")

We are ready to go. 

[In case you need it, here is a "Qiskit Cheat-Sheet" for you](http://keeptheketalive.fr/plus/Qiskit-Cheat-Sheet.pdf)

### 1 Building your first circuit

Let's build a circuit to generate first Bell State $\frac{1}{\sqrt{2}}(|00\rangle +|11\rangle)$

We will need 2 qubits to compute, and two cbits to read the result into, so we need to import `QuantumCircuit`, `QuantumRegister` and `ClassicalRegister`and construct instances of those.

Note about the first Bell state : 


<img src="./images/bell-state.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="350 px" align="center">



this is done using 2 qubits, system starts in state |00⟩ (both $q_0$ and $q_1$ in state |0⟩).
Then $q_0$ goes through Hadamard and gets into equal superposition of |0⟩ and |1⟩. 

Then $q_0$ and $q_1$ go through the CNOT gate (control not, the state of $q_0$ controls a NOT gate on $q_1$. 

After $q_0$ controls $q_1$, the state of $q_1$ is in a superposition of |0⟩ & |1⟩, ($q_1$ stays at |0⟩ when $q_0$ is |0⟩, and $q_1$ goes to |1⟩ when $q_0$ is |1⟩).

So : both $q_0$ and $q_1$ are in |0⟩ state (system is in state |00⟩) or both $q_0$ and $q_1$ are in |1⟩ (system is in state |11⟩).

Our system is in equal superposition of |00⟩ and |11⟩.

The two qubits are entangled together : if you measure any one of the qubits, you immediately know the state of the other !

### First import the needed elements

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

Then create the registers, of size 2, then the circuit using those registers (you may also create a circuit with implicit registers, there will be some examples later)

In [ ]:
qr = QuantumRegister(2,'q')
cr = ClassicalRegister(2,'c')

In [ ]:
qc = QuantumCircuit(qr,cr)

We can now "build" our circuit, by adding "gates" to the qubits: 

In this example we will need an hadamard gate on qubit 0, and a Cx gate from qubit 0 to qubit 1 

Gates will be method applied to the quantum circuit : 

`qc.h(qr[0])`

`qc.cx(qr[0],qr[1])`

You can refer to the documentation on qiskit.org, or type `dir(qc)` to view the valid methods on that object type. And then Python `help(obj.method)` gives you more information.


In [ ]:
qc = QuantumCircuit(qr,cr)
qc.h(qr[0])
qc.cx(qr[0],qr[1])

For small circuits, it can be very usefull to "draw" the circuit while you build it: 

`.draw()` accept `output=` styles : mpl, latex, latex_source, text(=default), ...

In [ ]:
qc.draw(output="mpl")

Before we execute, we need to make sure we perform a measurement at the end: 

Now we have a circuit, we are ready for execution , we still need : 
    
- to import `execute()` method,



- to decide where we want the circtui to be executed, we will call this a backend, it can be : the local simulator included with the qiskit library `qasm_simulator`, a simulator on the Cloud : `ibmq_qasm_simulator`or a real physical device on IBM Cloud as well, such as `ibmq_athens`. 

Let's first use the local simulator, for this we need to import Aer for simulator functions


   

### 2. Executing on local simulator

In [ ]:
#from qiskit_aer import AerSimulator
from qiskit import Aer

#print(Aer.backends()) #, nicer printout below:
backend_list  = Aer.backends()
for be in backend_list:
    print(be)


In [ ]:
from qiskit import execute

For now, we want to use `qasm_simulator` which emulates a physical backend, so let's get it : 

In [ ]:
backend = Aer.get_backend('qasm_simulator')

Now let's create our "job" to execute, `.execute()` requires that we pass the circuit and the backend (there are a number of optionnal argument, we will discuss "shots" soon : 

In [ ]:
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0,1)
qc.barrier()
qc.measure_all()
qc.draw()

In [ ]:
job = execute(qc,backend,shots=1000000)

In [ ]:
print(job)

Extracting results

In [ ]:
my_results = job.result()

In [ ]:
print(my_results)

Reading results :

In [ ]:
print(my_results.get_counts())

notes : 
- results are stored in a Python dictionnary, keys are the measurement basis, values are the number of times the measurement collapsed to that particular basis.
- the circuit has been run 1024 time, this is the default value for the keword parameter `shots`in `.execute()`. 

You can try values from `shots=1` to `shots=8192`

In [ ]:
# with shots=1, you will get a count of 1 either on 00 or on 11
job = execute(qc,backend, shots=1024)
my_results = job.result()
print(my_results.get_counts(qc))

It is also usefull to be able to visualize the results on a bar chart : 

In [ ]:
from qiskit.tools.visualization import plot_histogram
plot_histogram(my_results.get_counts(qc))

On more thing about reading results. We get used to enumerate qubits from top to bottom of a circuit, $q_0, q_1, ... , q_n$. The measurement will code multiple qubits in a binary register this way $q_n,..., q_1, q0$, in other words $q_0$ is the least significant bit. 


Let's see the following experiment, using $X$ (not) and $id$ (identity):  

In [ ]:
qr = QuantumRegister(3,'q')
cr = ClassicalRegister(3)
qc = QuantumCircuit(qr,cr)

qc.x(qr[0])
qc.x(qr[1])
qc.id(qr[2])

qc.measure(qr,cr)
qc.draw(output='mpl')

In this case we will measure a state corresponding to $q_0 = 1$ and $q_1= 0$, in the form $q_1q_0 = 01$

In [ ]:
# with shots=1, you will get a count of 1 either on 00 or on 11
job = execute(qc,backend, shots=1024)
my_results = job.result()
print(my_results.get_counts(qc))

### 3. Executing on statevector_simulator 

In [ ]:
import numpy as np
from qiskit import Aer, QuantumCircuit, execute
from qiskit.visualization import plot_histogram, plot_state_qsphere
from qiskit.quantum_info import Statevector
from IPython.display import display, Math, Latex

In [ ]:
#qc for bell state
qc = QuantumCircuit(2)

qc.h(0)
#qc.rx(np.pi/5,0)
qc.cx(0,1)
qc.rz(np.pi/7,1)

#qc.h(1)
qc.draw()

In [ ]:
sv = Statevector.from_label('00')
sv = sv.evolve(qc)

plot_state_qsphere(sv.data, show_state_labels=True, show_state_phases=True) 

In [ ]:
sv.data

In [ ]:
#qc for bell state
qc = QuantumCircuit(4)

qc.h(0)
qc.h(1)
qc.h(2)
qc.h(3)


qc.draw()

sv = Statevector.from_label('0'*4)
sv = sv.evolve(qc)

plot_state_qsphere(sv.data, show_state_labels=True, show_state_phases=True) 

In [ ]:
sv.data

### 4. Executing on a physical backend

Let's restart with the Bell State circuit 

Make sure you have installed qiskit_ibm_provider (pip install) 

In [ ]:
# Get your API token in https://quantum-computing.ibm.com/account
token = "....c207052b22c7f26d4a89a238ab305cf..."

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute

qc = QuantumCircuit(2,2)
qc.h(0)
qc.cx(0,1)
qc.measure([0,1],[0,1])
qc.draw()

`qc` is ready !

In [ ]:
from qiskit_ibm_provider import IBMProvider

provider = IBMProvider(token=token)
#backend = provider.get_backend("simulator_statevector")
backend = provider.get_backend("ibm_kyoto")



# Transpile circuit
from qiskit import transpile
transpiled_circuit = transpile(qc, backend)


# Run the circuit and get result
job = backend.run(transpiled_circuit)
counts = job.result().get_counts()
print(counts)

In [ ]:
be = provider.backends()
be

In [ ]:
ba = be[3]
print(ba.name)
dir(ba)

In [ ]:
print(help(ba.status))

In [ ]:
print(ba.operation_names)

In [ ]:
for _ in ba.instructions:
    print(_)